Neighborhoods in Toronto

In [30]:
import numpy as np 
import pandas as pd 
import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import matplotlib.cm as cm
import matplotlib.colors as colors # Matplotlib and associated plotting modules
from sklearn.cluster import KMeans # import k-means from clus



In [2]:
pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [31]:
from bs4 import BeautifulSoup # website scraping libraries and packages in Python from BeautifulSoup

In [6]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [32]:
from geopy.geocoders import Nominatim  # convert an address into latitude and longitude values

In [33]:
import folium # map rendering library

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

Download dataset

In [34]:
# GET request
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [45]:
#soup = BeautifulSoup(data, 'html.parser')



soup = BeautifulSoup(data, 'html.parser')
table = soup.find('table')
trs = table.find_all('tr')
rows = []
for tr in trs:
    i = tr.find_all('td')
    if i:
        rows.append(i)
        
lst = []
for row in rows:
    postalcode = row[0].text.rstrip()
    borough = row[1].text.rstrip()
    neighborhood = row[2].text.rstrip()
    if borough != 'Not assigned':
        if neighborhood == 'Not assigned':
            neighborhood = borough
        lst.append([postalcode, borough, neighborhood])

# lst


#postalCodeList = []
#boroughList = []
#neighborhoodList = []


    convert into a dataframe

In [49]:
cols = ['PostalCode', 'Borough', 'Neighborhood']
df = pd.DataFrame(lst, columns=cols)
print(df.shape)
df.head()  #Load dataframme

(103, 3)


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


custom groupby / agg to merge Neighborhoods

groupby PostalCode, keep the first Borough and join() Neighborhoods


In [48]:

df = df.groupby('PostalCode').agg(
    {
        'Borough':'first', 
        'Neighborhood': ', '.join,}
    ).reset_index()


will check if 'M5A' example is done correctly

In [50]:

df.loc[df['PostalCode'] == 'M5A']

,PostalCode,Borough,Neighborhood
2,M5A,Downtown Toronto,Regent Park / Harbourfront


    will read geo data from a CSV
    need to rename a column so the merge (below) will work

In [55]:
 #dfgeo=requests.get('http://cocl.us/Geospatial_data').text
dfgeo = pd.read_csv("Geospatial_Coordinates.csv")
dfgeo.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)
#dfgeo.head()

In [56]:
df2 = pd.merge(df, dfgeo, on="PostalCode", how='left') 
#need to merge 2 dataframes based on a PostalCode column

In [57]:
#let's check if the example returns the expected value(s)
df2.loc[df2['PostalCode'] == 'M5G']

,PostalCode,Borough,Neighborhood,Latitude,Longitude
24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383


Toronto Map

In [58]:
geolocator = Nominatim(user_agent="coursera")
address = 'Toronto'
try:
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    print('The geograpical coordinates of {} are {}, {}.'.format(address, latitude, longitude))
except AttributeError:
    print('Cannot find: {}, will drop index: {}'.format(address, index))

my_map = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df2['Latitude'], df2['Longitude'], df2['PostalCode']):
    label = folium.Popup(label)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(my_map)  
    
my_map

The geograpical coordinates of Toronto are 43.6534817, -79.3839347.


In [60]:
#Will analyze with 4Square


CLIENT_ID = '...' # Foursquare ID
CLIENT_SECRET = '...' # Foursquare Secret
VERSION = '20180605' # API version


#will focus on Etobicoke

df2.set_index('PostalCode', inplace = True) 
neighborhood_latitude = df2.loc['M9V']['Latitude']
neighborhood_longitude = df2.loc['M9V']['Longitude']

In [61]:

LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=...&client_secret=...&v=20180605&ll=43.739416399999996,-79.5884369&radius=500&limit=100'

In [62]:
results = requests.get(url).json()

In [63]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [66]:
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues) # flatten JSON

KeyError: 'groups'

In [69]:
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

NameError: name 'nearby_venues' is not defined

In [67]:


def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [68]:
df2

,Borough,Neighborhood,Latitude,Longitude
PostalCode,,,,
M3A,North York,Parkwoods,43.753259,-79.329656
M4A,North York,Victoria Village,43.725882,-79.315572
M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494
...,...,...,...,...
M8X,Etobicoke,The Kingsway / Montgomery Road / Old Mill North,43.653654,-79.506944
M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
M7Y,East Toronto,Business reply mail Processing CentrE,43.662744,-79.321558
